# Advent of code

## Day 1
--- Day 1: The Tyranny of the Rocket Equation ---

Santa has become stranded at the edge of the Solar System while delivering presents to other planets! To accurately calculate his position in space, safely align his warp drive, and return to Earth in time to save Christmas, he needs you to bring him measurements from fifty stars.

Collect stars by solving puzzles. Two puzzles will be made available on each day in the Advent calendar; the second puzzle is unlocked when you complete the first. Each puzzle grants one star. Good luck!

The Elves quickly load you into a spacecraft and prepare to launch.

At the first Go / No Go poll, every Elf is Go until the Fuel Counter-Upper. They haven't determined the amount of fuel required yet.

Fuel required to launch a given module is based on its mass. Specifically, to find the fuel required for a module, take its mass, divide by three, round down, and subtract 2.

For example:

    For a mass of 12, divide by 3 and round down to get 4, then subtract 2 to get 2.
    For a mass of 14, dividing by 3 and rounding down still yields 4, so the fuel required is also 2.
    For a mass of 1969, the fuel required is 654.
    For a mass of 100756, the fuel required is 33583.

The Fuel Counter-Upper needs to know the total fuel requirement. To find it, individually calculate the fuel needed for the mass of each module (your puzzle input), then add together all the fuel values.

What is the sum of the fuel requirements for all of the modules on your spacecraft?


In [1]:
from math import floor

In [22]:
def compute_fuel(mass):
    return floor((mass / 3.)) - 2

In [23]:
def compute_total_fuel(masses):
    total_fuel = 0
    for mass in masses:
        total_fuel += compute_fuel(mass)
    return total_fuel

In [24]:
masses = []
with open("day1p1.txt", "r", encoding='utf-8-sig') as file:
    for line in file:
        masses.append(int(line))
    
compute_total_fuel(masses)

3216744

During the second Go / No Go poll, the Elf in charge of the Rocket Equation Double-Checker stops the launch sequence. Apparently, you forgot to include additional fuel for the fuel you just added.

Fuel itself requires fuel just like a module - take its mass, divide by three, round down, and subtract 2. However, that fuel also requires fuel, and that fuel requires fuel, and so on. Any mass that would require negative fuel should instead be treated as if it requires zero fuel; the remaining mass, if any, is instead handled by wishing really hard, which has no mass and is outside the scope of this calculation.

So, for each module mass, calculate its fuel and add it to the total. Then, treat the fuel amount you just calculated as the input mass and repeat the process, continuing until a fuel requirement is zero or negative. For example:

    A module of mass 14 requires 2 fuel. This fuel requires no further fuel (2 divided by 3 and rounded down is 0, which would call for a negative fuel), so the total fuel required is still just 2.
    At first, a module of mass 1969 requires 654 fuel. Then, this fuel requires 216 more fuel (654 / 3 - 2). 216 then requires 70 more fuel, which requires 21 fuel, which requires 5 fuel, which requires no further fuel. So, the total fuel required for a module of mass 1969 is 654 + 216 + 70 + 21 + 5 = 966.
    The fuel required by a module of mass 100756 and its fuel is: 33583 + 11192 + 3728 + 1240 + 411 + 135 + 43 + 12 + 2 = 50346.

What is the sum of the fuel requirements for all of the modules on your spacecraft when also taking into account the mass of the added fuel? (Calculate the fuel requirements for each module separately, then add them all up at the end.)

In [34]:
def compute_total_fuel_single_module(mass):
    fuel_mass = compute_fuel(mass)
    total_fuel_mass = 0
    while fuel_mass > 0:
        total_fuel_mass += fuel_mass
        fuel_mass = compute_fuel(fuel_mass)
    return total_fuel_mass

In [35]:
def compute_total_modules_fuel(masses):
    total_fuel = 0
    for mass in masses:
        total_fuel += compute_total_fuel_single_module(mass)
    return total_fuel

In [36]:
masses = []
with open("day1p1.txt", "r", encoding='utf-8-sig') as file:
    for line in file:
        masses.append(int(line))
    
compute_total_modules_fuel(masses)

4822249

## Day 2
--- Day 2: 1202 Program Alarm ---

On the way to your gravity assist around the Moon, your ship computer beeps angrily about a "1202 program alarm". On the radio, an Elf is already explaining how to handle the situation: "Don't worry, that's perfectly norma--" The ship computer bursts into flames.

You notify the Elves that the computer's magic smoke seems to have escaped. "That computer ran Intcode programs like the gravity assist program it was working on; surely there are enough spare parts up there to build a new Intcode computer!"

An Intcode program is a list of integers separated by commas (like 1,0,0,3,99). To run one, start by looking at the first integer (called position 0). Here, you will find an opcode - either 1, 2, or 99. The opcode indicates what to do; for example, 99 means that the program is finished and should immediately halt. Encountering an unknown opcode means something went wrong.

Opcode 1 adds together numbers read from two positions and stores the result in a third position. The three integers immediately after the opcode tell you these three positions - the first two indicate the positions from which you should read the input values, and the third indicates the position at which the output should be stored.

For example, if your Intcode computer encounters 1,10,20,30, it should read the values at positions 10 and 20, add those values, and then overwrite the value at position 30 with their sum.

Opcode 2 works exactly like opcode 1, except it multiplies the two inputs instead of adding them. Again, the three integers after the opcode indicate where the inputs and outputs are, not their values.

Once you're done processing an opcode, move to the next one by stepping forward 4 positions.

For example, suppose you have the following program:

1,9,10,3,2,3,11,0,99,30,40,50

For the purposes of illustration, here is the same program split into multiple lines:

1,9,10,3,
2,3,11,0,
99,
30,40,50

The first four integers, 1,9,10,3, are at positions 0, 1, 2, and 3. Together, they represent the first opcode (1, addition), the positions of the two inputs (9 and 10), and the position of the output (3). To handle this opcode, you first need to get the values at the input positions: position 9 contains 30, and position 10 contains 40. Add these numbers together to get 70. Then, store this value at the output position; here, the output position (3) is at position 3, so it overwrites itself. Afterward, the program looks like this:

1,9,10,70,
2,3,11,0,
99,
30,40,50

Step forward 4 positions to reach the next opcode, 2. This opcode works just like the previous, but it multiplies instead of adding. The inputs are at positions 3 and 11; these positions contain 70 and 50 respectively. Multiplying these produces 3500; this is stored at position 0:

3500,9,10,70,
2,3,11,0,
99,
30,40,50

Stepping forward 4 more positions arrives at opcode 99, halting the program.

Here are the initial and final states of a few more small programs:

    1,0,0,0,99 becomes 2,0,0,0,99 (1 + 1 = 2).
    2,3,0,3,99 becomes 2,3,0,6,99 (3 * 2 = 6).
    2,4,4,5,99,0 becomes 2,4,4,5,99,9801 (99 * 99 = 9801).
    1,1,1,4,99,5,6,0,99 becomes 30,1,1,4,2,5,6,0,99.

Once you have a working computer, the first step is to restore the gravity assist program (your puzzle input) to the "1202 program alarm" state it had just before the last computer caught fire. To do this, before running the program, replace position 1 with the value 12 and replace position 2 with the value 2. What value is left at position 0 after the program halts?


In [43]:
from typing import List

class GravityIntcodeComputer:
    
    def __init__(self, program: List[int]):
        self.program = program
        self.cur_pos = 0
        self.is_finished = False
        
    def next_slice(self):
        next_slice = program[self.cur_pos : self.cur_pos + 4]
        self.cur_pos += 4
        if self.cur_pos > len(program):
            self.is_finished = True
        return next_slice

In [50]:
program = [1, 1, 1, 4, 99, 5, 6, 0, 99]
comp = GravityIntcodeComputer(program)
comp.next_slice()
comp.next_slice()
comp.next_slice()
comp.next_slice()

[]

## Day3
The gravity assist was successful, and you're well on your way to the Venus refuelling station. During the rush back on Earth, the fuel management system wasn't completely installed, so that's next on the priority list.

Opening the front panel reveals a jumble of wires. Specifically, two wires are connected to a central port and extend outward on a grid. You trace the path each wire takes as it leaves the central port, one wire per line of text (your puzzle input).

The wires twist and turn, but the two wires occasionally cross paths. To fix the circuit, you need to find the intersection point closest to the central port. Because the wires are on a grid, use the Manhattan distance for this measurement. While the wires do technically cross right at the central port where they both start, this point does not count, nor does a wire count as crossing with itself.

For example, if the first wire's path is R8,U5,L5,D3, then starting from the central port (o), it goes right 8, up 5, left 5, and finally down 3:

`...........
...........
...........
....+----+.
....|....|.
....|....|.
....|....|.
.........|.
.o-------+.
...........`

Then, if the second wire's path is U7,R6,D4,L4, it goes up 7, right 6, down 4, and left 4:
`
...........
.+-----+...
.|.....|...
.|..+--X-+.
.|..|..|.|.
.|.-X--+.|.
.|..|....|.
.|.......|.
.o-------+.
...........
`

These wires cross at two locations (marked X), but the lower-left one is closer to the central port: its distance is 3 + 3 = 6.

Here are a few more examples:

    R75,D30,R83,U83,L12,D49,R71,U7,L72
    U62,R66,U55,R34,D71,R55,D58,R83 = distance 159
    R98,U47,R26,D63,R33,U87,L62,D20,R33,U53,R51
    U98,R91,D20,R16,D67,R40,U7,R15,U6,R7 = distance 135

What is the Manhattan distance from the central port to the closest intersection?

In [104]:
class Path:
    
    def __init__(self, wire):
        self.xs = [0]
        self.ys = [0]
        
        for wire_str in wire:
            self.handle_wire_str(wire_str)
        
    def wire_len(self):
        return len(self.xs)
        
    def add_pos_to_path(self, x, y):
        self.xs.append(self.xs[-1] + x)
        self.ys.append(self.ys[-1] + y)

    def handle_wire_str(self, wire_str):
        direction = wire_str[0]
        distance = int(wire_str[1:])
        
        if direction == "U":
            self.add_pos_to_path(0, distance)
                
        elif direction == "D":
            self.add_pos_to_path(0, 0 - distance)
            
        elif direction == "R":
            self.add_pos_to_path(distance, 0)
            
        elif direction == "L":
            self.add_pos_to_path(0 - distance, 0)
    

In [117]:
class Segment:
    
    def __init__(self, x1, y1, x2, y2):
        if x1 < x2:
            self.x1 = x1
            self.x2 = x2
        else:
            self.x1 = x2
            self.x2 = x1
        if y1 < y2:
            self.y1 = y1
            self.y2 = y2
        else:
            self.y1 = y2
            self.y2 = y1
            
        self.is_vert = x1 == x2
        
    def get_intersection(self, seg):
        if self.is_vert == seg.is_vert:
            return (0,0)
        if self.is_vert:
            if seg.y1 >= self.y1 and seg.y1 <= self.y2 and self.x1 >= seg.x1 and self.x1 <= seg.x2:
                return (self.x1, seg.y1)
        else:
            if self.y1 >= seg.y1 and self.y1 <= seg.y2 and seg.x1 >= self.x1 and seg.x1 <= self.x2:
                return (seg.x1, self.y1)
        return (0,0)

In [127]:
def manhattan_distance(intersection):
    return abs(intersection[0]) + abs(intersection[1])

In [128]:
def find_smallest_distance(path1, path2):
    smallest_distance = float('Inf')
    for i in range(path1.wire_len() - 1):
        seg1 = Segment(path1.xs[i], path1.ys[i], path1.xs[i+1], path1.ys[i+1])
        for j in range(path2.wire_len() - 1):
            seg2 = Segment(path2.xs[j], path2.ys[j], path2.xs[j+1], path2.ys[j+1])
            intersection = seg1.get_intersection(seg2)
            if intersection == (0,0):
                continue
            dist = manhattan_distance(intersection)
            if dist < smallest_distance:
                smallest_distance = dist
                
    return smallest_distance

In [129]:
#wire_strs1 = ["R8","U5","L5","D3"]
#wire_strs2 = ["U7","R6","D4","L4"]

#wire_strs1 = ["R75","D30","R83","U83","L12","D49","R71","U7","L72"]
#wire_strs2 = ["U62","R66","U55","R34","D71","R55","D58","R83"]

wire_strs1 = ["R98","U47","R26","D63","R33","U87","L62","D20","R33","U53","R51"]
wire_strs2 = ["U98","R91","D20","R16","D67","R40","U7","R15","U6","R7"]

pa1 = Path(wire_strs1)
pa2 = Path(wire_strs2)


find_smallest_distance(pa1, pa2)

135

In [130]:
wires = []
with open("day3.txt", "r", encoding='utf-8-sig') as file:
    for line in file:
        line = line.strip()
        wires.append(line.split(','))
        
path1 = Path(wires[0])
path2 = Path(wires[1])

In [131]:
find_smallest_distance(path1, path2)

5319

--- Part Two ---

It turns out that this circuit is very timing-sensitive; you actually need to minimize the signal delay.

To do this, calculate the number of steps each wire takes to reach each intersection; choose the intersection where the sum of both wires' steps is lowest. If a wire visits a position on the grid multiple times, use the steps value from the first time it visits that position when calculating the total value of a specific intersection.

The number of steps a wire takes is the total number of grid squares the wire has entered to get to that location, including the intersection being considered. Again consider the example from above:

`
...........
.+-----+...
.|.....|...
.|..+--X-+.
.|..|..|.|.
.|.-X--+.|.
.|..|....|.
.|.......|.
.o-------+.
...........
`

In the above example, the intersection closest to the central port is reached after 8+5+5+2 = 20 steps by the first wire and 7+6+4+3 = 20 steps by the second wire for a total of 20+20 = 40 steps.

However, the top-right intersection is better: the first wire takes only 8+5+2 = 15 and the second wire takes only 7+6+2 = 15, a total of 15+15 = 30 steps.

Here are the best steps for the extra examples from above:

    R75,D30,R83,U83,L12,D49,R71,U7,L72
    U62,R66,U55,R34,D71,R55,D58,R83 = 610 steps
    R98,U47,R26,D63,R33,U87,L62,D20,R33,U53,R51
    U98,R91,D20,R16,D67,R40,U7,R15,U6,R7 = 410 steps

What is the fewest combined steps the wires must take to reach an intersection?


In [132]:
class Segment_len(Segment):
    def __init__(self, x1, y1, x2, y2):
        super(Segment_len, self).__init__(x1, y1, x2, y2)
        self.len: int
        if self.is_vert:
            self.len = self.y2 - self.y1
        else:
            self.len = self.x2 - self.x1

## Day4

--- Day 4: Secure Container ---

You arrive at the Venus fuel depot only to discover it's protected by a password. The Elves had written the password on a sticky note, but someone threw it out.

However, they do remember a few key facts about the password:

    It is a six-digit number.
    The value is within the range given in your puzzle input.
    Two adjacent digits are the same (like 22 in 122345).
    Going from left to right, the digits never decrease; they only ever increase or stay the same (like 111123 or 135679).

Other than the range rule, the following are true:

    111111 meets these criteria (double 11, never decreases).
    223450 does not meet these criteria (decreasing pair of digits 50).
    123789 does not meet these criteria (no double).

How many different passwords within the range given in your puzzle input meet these criteria?

Your puzzle input is 357253-892942.

In [52]:
start = 357253
end = 892942
valids = 0


In [51]:
n = 0
for i in range(start, end):
    n += 1
n

535689

In [53]:
for i in range(start, end):
    is_valid = False
    
    number = str(i)
    
    #2 numbers pareils
    for j in range(5):
        if number[j] == number[j+1]:
            is_valid = True
            break
            
    for j in range(5):
        if int(number[j]) > int(number[j+1]):
            is_valid = False
            break
    
    if is_valid:
        valids += 1

In [54]:
valids

530

--- Part Two ---

An Elf just remembered one more important detail: the two adjacent matching digits are not part of a larger group of matching digits.

Given this additional criterion, but still ignoring the range rule, the following are now true:

    112233 meets these criteria because the digits never decrease and all repeated digits are exactly two digits long.
    123444 no longer meets the criteria (the repeated 44 is part of a larger group of 444).
    111122 meets the criteria (even though 1 is repeated more than twice, it still contains a double 22).

How many different passwords within the range given in your puzzle input meet all of the criteria?


In [85]:
start = 357253
end = 892942
valids = 0

In [86]:
for i in range(start, end):
    is_valid = False
    
    number = str(i)
    
    #2 numbers pareils
    for j in range(5):
        pair = (number[j], number[j+1])
        if pair[0] == pair[1]:
            num = number[j]
            if (j == 0 or number[j-1] != num):
                if (j == 4 or number[j+2] != num):
                    is_valid = True
                    break
            
    for j in range(5):
        if int(number[j]) > int(number[j+1]):
            is_valid = False
            break
    
    if is_valid:
        valids += 1

In [87]:
valids

324

## Day 6

--- Day 6: Universal Orbit Map ---

You've landed at the Universal Orbit Map facility on Mercury. Because navigation in space often involves transferring between orbits, the orbit maps here are useful for finding efficient routes between, for example, you and Santa. You download a map of the local orbits (your puzzle input).

Except for the universal Center of Mass (COM), every object in space is in orbit around exactly one other object. An orbit looks roughly like this:

`
.
                  \
                   \
                    |
                    |
AAA--> o            o <--BBB
                    |
                    |
                   /
                  /
`

In this diagram, the object BBB is in orbit around AAA. The path that BBB takes around AAA (drawn with lines) is only partly shown. In the map data, this orbital relationship is written AAA)BBB, which means "BBB is in orbit around AAA".

Before you use your map data to plot a course, you need to make sure it wasn't corrupted during the download. To verify maps, the Universal Orbit Map facility uses orbit count checksums - the total number of direct orbits (like the one shown above) and indirect orbits.

Whenever A orbits B and B orbits C, then A indirectly orbits C. This chain can be any number of objects long: if A orbits B, B orbits C, and C orbits D, then A indirectly orbits D.

For example, suppose you have the following map:

`
COM)B
B)C
C)D
D)E
E)F
B)G
G)H
D)I
E)J
J)K
K)L
`

Visually, the above map of orbits looks like this:

`
.
        G - H       J - K - L
       /           /
COM - B - C - D - E - F
               \
                I
`

In this visual representation, when two objects are connected by a line, the one on the right directly orbits the one on the left.

Here, we can count the total number of orbits as follows:

    D directly orbits C and indirectly orbits B and COM, a total of 3 orbits.
    L directly orbits K and indirectly orbits J, E, D, C, B, and COM, a total of 7 orbits.
    COM orbits nothing.

The total number of direct and indirect orbits in this example is 42.



What is the total number of direct and indirect orbits in your map data?

In [1]:
orbits = []
with open("day6.txt", "r") as file:
    for line in file:
        line = line.strip()
        orbits.append((line.split(')')[0], line.split(')')[1]))
orbits

[('HL2', 'HP5'),
 ('PD5', 'SVL'),
 ('GGS', 'YTS'),
 ('59G', 'C67'),
 ('V3R', 'BTY'),
 ('B9B', '4QH'),
 ('P42', 'PNP'),
 ('3MR', 'YPD'),
 ('HPL', 'T7M'),
 ('9T9', 'BV2'),
 ('QTG', 'LZM'),
 ('LYL', '2K1'),
 ('ZM2', 'CCK'),
 ('983', 'QLL'),
 ('WYK', 'SKS'),
 ('G2H', '5QT'),
 ('GW2', '39Q'),
 ('K68', 'HBF'),
 ('FLF', 'GYC'),
 ('Y8H', 'B69'),
 ('H61', 'TSN'),
 ('J6L', 'PZH'),
 ('BD3', 'WSD'),
 ('321', 'HZ7'),
 ('FC4', 'X58'),
 ('7M7', 'SR7'),
 ('KXX', 'S7P'),
 ('JBT', 'HFR'),
 ('DPQ', '96V'),
 ('47S', '36P'),
 ('JQ8', 'FJL'),
 ('1PX', 'J2L'),
 ('DR8', 'HJ5'),
 ('1P6', 'GS4'),
 ('LXJ', 'WDK'),
 ('8BB', 'LYL'),
 ('TPD', '9XH'),
 ('5ZB', 'TML'),
 ('ZYQ', 'DLC'),
 ('1D2', 'L82'),
 ('SZX', 'TLH'),
 ('3HG', '2Z4'),
 ('T16', 'DT9'),
 ('TDS', '8PD'),
 ('Y9F', 'VNZ'),
 ('7NW', 'LQD'),
 ('F5W', '88Z'),
 ('YQH', 'N7L'),
 ('TRH', 'V7C'),
 ('FHB', 'WDL'),
 ('DT3', 'BHQ'),
 ('4Q8', 'MXM'),
 ('J1X', '5DP'),
 ('172', '6NG'),
 ('SR7', 'FLL'),
 ('DT9', 'PZP'),
 ('M8V', 'FY6'),
 ('RXG', 'Y4B'),
 ('H5F', '9G9'

In [2]:
to_visit = ['COM']
next_visit_round = []
total_orbits = 0
cur_depth = 1
while to_visit:
    for orbit_to_visit in to_visit:
        for orbit in orbits:
            if orbit[0] == orbit_to_visit:
                next_visit_round.append(orbit[1])
                total_orbits += cur_depth
    to_visit = next_visit_round
    next_visit_round = []
    cur_depth += 1

total_orbits

145250

--- Part Two ---

Now, you just need to figure out how many orbital transfers you (YOU) need to take to get to Santa (SAN).

You start at the object YOU are orbiting; your destination is the object SAN is orbiting. An orbital transfer lets you move from any object to an object orbiting or orbited by that object.

For example, suppose you have the following map:

COM)B
B)C
C)D
D)E
E)F
B)G
G)H
D)I
E)J
J)K
K)L
K)YOU
I)SAN

Visually, the above map of orbits looks like this:
`.
                          YOU
                         /
        G - H       J - K - L
       /           /
COM - B - C - D - E - F
               \
                I - SAN
`

In this example, YOU are in orbit around K, and SAN is in orbit around I. To move from K to I, a minimum of 4 orbital transfers are required:

    K to J
    J to E
    E to D
    D to I

Afterward, the map of orbits looks like this:

`.
        G - H       J - K - L
       /           /
COM - B - C - D - E - F
               \
                I - SAN
                 \
                  YOU
                  `
                  

What is the minimum number of orbital transfers required to move from the object YOU are orbiting to the object SAN is orbiting? (Between the objects they are orbiting - not between YOU and SAN.)


In [ ]:
#bfs


## Day 8

--- Day 8: Space Image Format ---

The Elves' spirits are lifted when they realize you have an opportunity to reboot one of their Mars rovers, and so they are curious if you would spend a brief sojourn on Mars. You land your ship near the rover.

When you reach the rover, you discover that it's already in the process of rebooting! It's just waiting for someone to enter a BIOS password. The Elf responsible for the rover takes a picture of the password (your puzzle input) and sends it to you via the Digital Sending Network.

Unfortunately, images sent via the Digital Sending Network aren't encoded with any normal encoding; instead, they're encoded in a special Space Image Format. None of the Elves seem to remember why this is the case. They send you the instructions to decode it.

Images are sent as a series of digits that each represent the color of a single pixel. The digits fill each row of the image left-to-right, then move downward to the next row, filling rows top-to-bottom until every pixel of the image is filled.

Each image actually consists of a series of identically-sized layers that are filled in this way. So, the first digit corresponds to the top-left pixel of the first layer, the second digit corresponds to the pixel to the right of that on the same layer, and so on until the last digit, which corresponds to the bottom-right pixel of the last layer.

For example, given an image 3 pixels wide and 2 pixels tall, the image data 123456789012 corresponds to the following image layers:

Layer 1: 123
         456

Layer 2: 789
         012

The image you received is 25 pixels wide and 6 pixels tall.

To make sure the image wasn't corrupted during transmission, the Elves would like you to find the layer that contains the fewest 0 digits. On that layer, what is the number of 1 digits multiplied by the number of 2 digits?


In [11]:
nums = []
with open("day8.txt", "r") as file:
    for line in file:
        line = line.strip()
        for i in line:
            nums.append(i)
            
nums

['2',
 '2',
 '2',
 '2',
 '0',
 '0',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '1',
 '2',
 '0',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '0',
 '2',
 '2',
 '2',
 '2',
 '0',
 '2',
 '0',
 '2',
 '0',
 '0',
 '2',
 '2',
 '1',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '0',
 '2',
 '2',
 '2',
 '2',
 '2',
 '1',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '0',
 '1',
 '2',
 '2',
 '0',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '1',
 '0',
 '2',
 '2',
 '2',
 '2',
 '2',
 '0',
 '2',
 '2',
 '2',
 '1',
 '2',
 '1',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '0',
 '2',
 '2',
 '1',
 '0',
 '1',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '1',
 '2',
 '2',
 '2',
 '2',
 '2',
 '0',
 '2',
 '2',
 '2',
 '0',
 '1',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '1',
 '2'

In [12]:
len(nums)

15000

In [15]:
n_per_item = int(len(nums)/(6*25))
n_per_item

100

In [22]:
liner_image = []
for i in range(6*25):
    liner_image.append(''.join(nums[i*n_per_item:(i+1)*n_per_item]))
liner_image

['2222002222221202222222222222222202222020200221222222222222202222212222222201220222222222222222222102',
 '2222022212122222202210122222222222222222222122222022201222222222212222222222222222022220202212222222',
 '2222222221222220221222220122222222222222222222201222220222222222222022111222222222222222222222222221',
 '2221002222220220222222222222222212222120202221222222222222222222212212222202221222222222222222222202',
 '2222122222222222222212122222222222222222222122222122222022222212012222222222222222222220212002202222',
 '2222222222222220221222222222022222222222222222212222222222022222222122112222222222222222222221222222',
 '2221102222221110222222222222222202222122200222222222222222212212212222222222221222222222222222222102',
 '2222222212122222222202022222222222222222222022222222210022222211202222222222222222222221212012202222',
 '2222222222220222221222221122222222222222222222201222221222220222222022121222222222202222222222212221',
 '22201222222211112222222222222222022221222102222222222

In [23]:
image = []
for i in range(6):
    image.append(liner_image[i*25: (i+1)*25])
image

[['2222002222221202222222222222222202222020200221222222222222202222212222222201220222222222222222222102',
  '2222022212122222202210122222222222222222222122222022201222222222212222222222222222022220202212222222',
  '2222222221222220221222220122222222222222222222201222220222222222222022111222222222222222222222222221',
  '2221002222220220222222222222222212222120202221222222222222222222212212222202221222222222222222222202',
  '2222122222222222222212122222222222222222222122222122222022222212012222222222222222222220212002202222',
  '2222222222222220221222222222022222222222222222212222222222022222222122112222222222222222222221222222',
  '2221102222221110222222222222222202222122200222222222222222212212212222222222221222222222222222222102',
  '2222222212122222222202022222222222222222222022222222210022222211202222222222222222222221212012202222',
  '2222222222220222221222221122222222222222222222201222221222220222222022121222222222202222222222212221',
  '2220122222221111222222222222222202222122210

In [25]:
layers = [''.join(x) for x in image]
layers

['22220022222212022222222222222222022220202002212222222222222022222122222222012202222222222222222221022222022212122222202210122222222222222222222122222022201222222222212222222222222222022220202212222222222222222122222022122222012222222222222222222220122222022222222222202211122222222222222222222222222122210022222202202222222222222222122221202022212222222222222222222122122222022212222222222222222222022222122222222222222212122222222222222222222122222122222022222212012222222222222222222220212002202222222222222222222022122222222202222222222222222221222222222202222222212211222222222222222222222122222222211022222211102222222222222222022221222002222222222222222122122122222222222212222222222222222221022222222212122222222202022222222222222222222022222222210022222211202222222222222222222221212012202222222222222222022222122222112222222222222222222220122222122222022222202212122222222220222222222221222122201222222211112222222222222222022221222102222222222222222022122222122222102222222222222222222222

In [31]:
def split(word): 
    return [char for char in word]  

min_zeros = 10000000000
min_ = 9
for i in range(6):
    z = split(layers[i]).count('0')
    if z < min_zeros:
        min_ = i
        min_zeros = z
min_

0

In [33]:
n_1 = 0
n_2 = 0

for i in split(layers[0]):
    if i == '1':
        n_1 +=1
    elif i == '2':
        n_2 +=1
n_1*n_2

442889

## Day 9